In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.data.sampler

In [2]:
class MNIST:
    """Dataset of 42000 28x28 grayscale images of handwritten characters"""

    image_shape = (1, 28, 28)
    num_classes = 10

    @classmethod
    def _get_images(cls, df):
        x = df.values
        x = x.reshape((-1, *cls.image_shape))
        x = x.astype(np.float32)
        x /= 255
        return x

    def __init__(self, input_file, train=True):

        df = pd.read_csv(input_file)

        if train:
            self.labels = df['label'].values
            df = df.drop(columns=['label'])

        self.images = self._get_images(df)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if hasattr(self, 'labels'):
            return self.images[idx, :], self.labels[idx]
        else:
            return self.images[idx, :]

In [3]:
class DigitRecognizerCNN(nn.Module):
    """Simple convolutional network consisting of 2 convolution layers with
    max-pooling followed by two fully-connected layers and a softmax output
    layer
    """

    def __init__(self, num_classes):

        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(32 * 7 * 7, 784),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(784, 784),
            nn.ReLU(),
            nn.Linear(784, num_classes))

    def forward(self, X):
        x = self.features(X)
        x = x.view(x.size(0), 32 * 7 * 7)
        x = self.classifier(x)
        return x

In [4]:
model = DigitRecognizerCNN(num_classes=MNIST.num_classes)

parameters = torch.load('kernel_cnn.pth')

model.load_state_dict(parameters)

In [5]:
test_data = MNIST('C:/Users/dell/Desktop/mnist/test.csv', train=False)
images = torch.Tensor(test_data.images)
logits = model.to('cpu')(images)
_, y_pred = torch.max(logits, dim=1)

submission = pd.DataFrame({'ImageId': np.arange(len(y_pred)) + 1, 'Label': y_pred})
submission.head(10)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,7
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3
